In [1]:
import os 
import numpy as np
import pandas as pd

In [5]:
folder_path = 'pSERS'
output_csv = 'pSERS.csv'
common_axis = np.linspace(178, 2445, 1024)

txt_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.txt')])
data = []
labels = []

for file in txt_files:
    path = os.path.join(folder_path, file)
    try:
        df = pd.read_csv(path, delim_whitespace=True, header=None)
        wn = df.iloc[:, 0].values
        intensity = df.iloc[:, 1].values

        # Crop to target range
        mask = (wn > 178) & (wn < 2455)
        wn_crop = wn[mask]
        intensity_crop = intensity[mask]

        # Interpolate to common axis
        intensity_interp = np.interp(common_axis, wn_crop, intensity_crop)

        data.append(intensity_interp)
        label = os.path.splitext(file)[0].split('_')[0]
        labels.append(label)

    except Exception as e:
        print(f"⚠️ Error with file {file}: {e}")
        data.append(np.full_like(common_axis, np.nan))
        labels.append("Error")

# Final DataFrame
df_out = pd.DataFrame(data, columns=common_axis)
df_out['Label'] = labels
df_out.to_csv(output_csv, index=False)

print(f"✅ Saved aligned query set with shape {df_out.shape} to '{output_csv}'.")


C:\Users\levinej\AppData\Local\Temp\ipykernel_24500\2720741107.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(path, delim_whitespace=True, header=None)
C:\Users\levinej\AppData\Local\Temp\ipykernel_24500\2720741107.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(path, delim_whitespace=True, header=None)
C:\Users\levinej\AppData\Local\Temp\ipykernel_24500\2720741107.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(path, delim_whitespace=True, header=None)
C:\Users\levinej\AppData\Local\Temp\ipykernel_24500\2720741107.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future vers

✅ Saved aligned query set with shape (75, 1025) to 'pSERS.csv'.


In [8]:
import pandas as pd

# 1) Define your source files and their substrate labels
files = {
    'pSERS.csv': 'pSERS',
    'PICO.csv': 'PICO',
    'AgNP.csv': 'AgNP',
    'AuNP.csv': 'AuNP',
    'Ag.csv': 'Ag',
    'Au.csv': 'Au'
}

# 2) Read, annotate, and collect
df_list = []
for fname, substrate in files.items():
    df = pd.read_csv(f'{fname}')  # adjust path if needed
    df['Substrate'] = substrate
    df_list.append(df)

# 3) Concatenate into one DataFrame
consolidated = pd.concat(df_list, ignore_index=True)

# 4) Save the consolidated dataset
consolidated.to_csv('consolidated_SERS.csv', index=False)


In [16]:
consolidated_path = 'consolidated_SERS.csv'
df = pd.read_csv(consolidated_path)

spectral_cols = [c for c in df.columns if c not in ['Label', 'Substrate']]
avg_df = (df.groupby(['Label', 'Substrate'])[spectral_cols].mean().reset_index())
avg_df = avg_df[spectral_cols + ['Label', 'Substrate']]

out_path = 'consolidated_SERS_avg.csv'
avg_df.to_csv(out_path, index=False)